In [ ]:
from bs4 import BeautifulSoup
import requests
from googletrans import Translator
from transformers import pipeline, BartTokenizer
import sacrebleu
from rouge_score import rouge_scorer


def fetch_article(url):
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    article_text = ' '.join([p.text for p in soup.find_all('p')])
    return article_text

def translate_text(text, dest_lang='en'):
    translator = Translator()
    translation = translator.translate(text, dest=dest_lang)
    return translation.text

def evaluate_translation(hypotheses, references):
    bleu_score = sacrebleu.raw_corpus_bleu(hypotheses, [references]).score
    return bleu_score

def summarize_text(text):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
    max_length = tokenizer.model_max_length
    if tokens.input_ids.size(1) <= max_length:
        summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
        summary_text = summary[0]['summary_text']
    else:
        part_size = max_length - 50
        parts = [text[i:i + part_size] for i in range(0, len(text), part_size)]
        summary_text = ''
        for part in parts:
            summary_part = summarizer(part, max_length=130, min_length=30, do_sample=False)
            summary_text += summary_part[0]['summary_text'] + ' '
    return summary_text

def evaluate_summary(hypotheses, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(references, hypotheses)
    return scores

def main():
    print("Please enter the URL of the Arabic news article:")
    url = input()
    print("Fetching and processing the article...")
    article_text = fetch_article(url)
    translated_text = translate_text(article_text, dest_lang='en')

    # Example Reference Texts (For demonstration; replace with actual reference texts)
    reference_translation = "Example reference translation text."
    reference_summary = "Example reference summarization text."

    # Evaluate Translation
    translation_score = evaluate_translation([translated_text], [reference_translation])
    print("Translation BLEU Score:",translation_score )

    summary_text = summarize_text(translated_text)
    summary_scores = evaluate_summary(summary_text, reference_summary)
    print("Summary ROUGE Scores:",summary_scores)

    print("Translation:\n", translated_text)
    print("Summary:\n", summary_text)

if __name__ == "__main__":
    main()

Please enter the URL of the Arabic news article:
https://www.aljazeera.net/politics/2024/5/3/%D8%B9%D8%A7%D8%A6%D9%84%D8%A9-%D9%85%D8%B9%D8%A7%D8%B1%D8%B6-%D8%AA%D9%88%D9%86%D8%B3%D9%8A-%D9%85%D8%B6%D8%B1%D8%A8-%D8%B9%D9%86-%D8%A7%D9%84%D8%B7%D8%B9%D8%A7%D9%85-%D8%A7%D8%A8%D9%86%D9%86%D8%A7
Fetching and processing the article...
Translation BLEU Score:0.6


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Summary ROUGE Scores:0.5
Translation:
 TUNIS- The family of the detained Tunisian opposition, Johar bin Mubarak, is experiencing great concern that increased its severity after his health deteriorated after his constant hunger strike 10 days ago in protest against the passage of a year since his arrest with the rest of the political detainees, due to their opposition to the path of July 25, 2021 imposed by President Qais Saeed.Dalila bin Mubarak Mossadak, the sister of Johar bin Mubarak and a member of the Defense Authority for the Political Detainees, has made the alarm about the health of her brother who was transferred to the hospital due to the deterioration of his health, but he refused to suspend his hunger strike, water and medicine.In an interview with Al -Jazeera Net, Delilah said that the hospital's essence was deposited due to severe kidney pain, and after the analyzes it was found that he had blood clotting, meaning that he entered the stage of danger to his life, and he re

In [ ]:
pip install --upgrade googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=679d1531bd20aacce3b4e0045550f8fa3253bd877f8cbf0d3aba7417e157d3ae
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=0d2a6ed567a545237f11af2202a35e3d5fbdd9505b58a2fb928cc04a58f7aba1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.2 MB/s eta 0:00:00
